In [1]:
import sys
print(sys.path)

['/home/beidan/AL-word2vec', '/home/beidan/miniconda3/lib/python38.zip', '/home/beidan/miniconda3/lib/python3.8', '/home/beidan/miniconda3/lib/python3.8/lib-dynload', '', '/home/beidan/.local/lib/python3.8/site-packages', '/home/beidan/miniconda3/lib/python3.8/site-packages', '/home/beidan/miniconda3/lib/python3.8/site-packages/IPython/extensions', '/home/beidan/.ipython']


In [2]:
## Save a smaller patentdb input
# aaa = []
# with open('/home/beidan/AutoPhrase/data/EN/patentdb.txt') as f:
#     i=0
#     for line in f:
#         if(len(line)<5):continue
#         line=line.replace('\n','')
#         if i>5000: continue
#         aaa.append(line)
#         i+=1

In [3]:
# with open('/home/beidan/AutoPhrase/data/EN/patentdb-medium.txt', 'w') as f:
#     f.writelines("%s\n" % line for line in aaa)

In [4]:
# with open('/home/beidan/AutoPhrase/models/DBLP/segmentation.txt') as content:
#     i=0
#     with open('patentdb_seg_1K','w') as f:
#         for line in content:
#             if(i>2000):continue
#             if(len(line)<5):continue
#             f.write(line)
#             i+=1

### Preparing the model

In [ ]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
import re

In [10]:
## Prepare Formatted Corpus

In [11]:
def process_tag_strings(tag_str):
    pattern = '<phrase>(.*?)</phrase>'
    matches = re.findall(pattern, tag_str, re.S)  # This returns a list of all matches
    for match in matches:
#         str_to_replace = match.group(1)  # 1 index is the string in between the tags
        tag_str = tag_str.replace(match, match.strip().replace(' ', '_'))
    return tag_str

In [12]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams_embeded(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return ['_'.join(grams) for grams in n_grams]


def convert_ngram(line, n):
    return ' '.join(get_ngrams_embeded(line, n))
    

In [13]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [14]:
from bs4 import BeautifulSoup

def get_quality_phrases(lines):
    quality_phrases=set()
    for line in lines:
        soup = BeautifulSoup(line, 'html.parser')
        phrases = soup.findAll('phrase')
        for p in phrases:
            p = p.text
            quality_phrases.add(p)
    return quality_phrases

In [15]:
from bs4 import BeautifulSoup
original_corpus=[]
def get_raw_corpus():
      with open('input/segmentation.txt') as content:
        for line in content:
            if(len(line)<5):continue
            line=line.replace('\n','')
            original_corpus.append(line)  
            
get_raw_corpus()

In [16]:
len(original_corpus)

5001

In [50]:
train_len = round(len(original_corpus) * 0.85)

In [51]:
quality_phrases_train_set = get_quality_phrases(original_corpus[:train_len])
quality_phrases_test_set = get_quality_phrases(original_corpus[train_len+1:])

In [52]:
quality_phrases_train = [p.replace(' ', '_') for p in (quality_phrases_train_set)]
quality_phrases_train = [p.lower() for p in (quality_phrases_train)]

In [53]:
quality_phrases_test = [p.replace(' ', '_') for p in (quality_phrases_test_set)]
quality_phrases_test = [p.lower() for p in (quality_phrases_test)]

In [54]:
quality_phrases_train[:5]

['identity_registration_database',
 'trail',
 'musical_instrument',
 'personalization',
 'drive_source']

In [36]:
articles = []
lines = []

def get_corpus():
    with open('input/segmentation.txt') as content:
        for line in content:
            if(len(line)<5):continue
#             line=process_tag_strings(str(line))
            #senario 2: convert to all bi-grams
            line=cleanhtml(line)
            line = convert_ngram(line, 2)
            lines.append(line)
            line=line.replace('\n','')
            articles.append(line)
            
get_corpus()

In [10]:
unprocessed_lines=[]

def get_corpus_unprocessed():
    with open('input/segmentation.txt') as content:
        for line in content:
            if(len(line)<5):continue
#             line=process_tag_strings(str(line))
            lines.append(line)
            line=cleanhtml(line)
            line=line.replace('\n','')
            unprocessed_lines.append(line)
            
get_corpus_unprocessed()

In [11]:
len(articles)

20971

In [9]:
### Create the dataframe

In [10]:
# df = pd.DataFrame(data=np.array(articles), columns=["text"])

### Build the word2Vec Model

In [37]:
def build_vocab(sentences):
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv

In [38]:
def get_embeddings(inp_data, vocabulary_inv, size_features=100,
                   mode='cbow',
                   min_word_count=5,
                   context=5):
    model_name = "embedding"
    model_name = os.path.join(model_name)
    num_workers = 15  # Number of threads to run in parallel
    downsampling = 1e-3  # Downsample setting for frequent words
    print('Training Word2Vec model...')
    sentences = [[vocabulary_inv[w] for w in s] for s in inp_data]
    if mode == 'skipgram':
        sg = 1
        print('Model: skip-gram')
    elif mode == 'cbow':
        sg = 0
        print('Model: CBOW')
    embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                        sg=sg,
                                        size=size_features,
                                        min_count=min_word_count,
                                        window=context)
#                                         sample=downsampling)
    embedding_model.init_sims(replace=True)
    print("Saving Word2Vec model {}".format(model_name))
#     embedding_weights = np.zeros((len(vocabulary_inv), size_features))
#     for i in range(len(vocabulary_inv)):
#         word = vocabulary_inv[i]
#         if word in embedding_model:
#             embedding_weights[i] = embedding_model[word]
#         else:
# #             print("word not in model: ", word)
#             embedding_weights[i] = np.random.uniform(-0.25, 0.25,
#                                                      embedding_model.vector_size)
    return embedding_model

In [39]:
punc_excluded = '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'

def preprocess_df(df):
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    translator = str.maketrans(punc_excluded, ' ' * len(punc_excluded))
    preprocessed_sentences = []
    for row in df:
        sent = row
        sent_nopuncts = sent.translate(translator)
        words_list = sent_nopuncts.strip().lower().split(' ')
        filtered_words = [word for word in words_list if word not in stop_words and len(word) != 1]
        preprocessed_sentences.append(" ".join(filtered_words))
#     df["text"] = preprocessed_sentences
    return preprocessed_sentences

In [41]:
df_train = articles[:train_len]
df_train = preprocess_df(df_train)
df_test = articles[train_len+1:]
df_test = preprocess_df(df_test)

In [42]:


tagged_data = [word_tokenize(_d) for i, _d in enumerate(df_train)]
word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_data)
inp_data = [[vocabulary[word] for word in text] for text in tagged_data]
embedding_model = get_embeddings(inp_data, vocabulary_inv)


tagged_train_data = [word_tokenize(_d) for i, _d in enumerate(df_train)]
# tagged_test_data = [word_tokenize(_d) for i, _d in enumerate(df_test)]



Training Word2Vec model...
Model: CBOW
Saving Word2Vec model embedding


In [43]:
tagged_data[0]

['kr20190120984a_',
 '_the',
 'the_self-supporting',
 'self-supporting_unmanned',
 'unmanned_border',
 'border_system',
 'system_according',
 'according_to',
 'to_an',
 'an_embodiment',
 'embodiment_of',
 'of_the',
 'the_present',
 'present_invention',
 'invention_utilizes',
 'utilizes_radar',
 'radar_and',
 'and_the',
 'the_internet',
 'internet_of',
 'of_things',
 'things_']

In [250]:
# with open('tagged_data.txt','w') as f:
#     for item in tagged_data:
#         f.write("%s\n" % item)

In [44]:
embedding_idx_to_word=dict()

def get_quality_phrases_embeddings(quality_phrases):
    quality_embeddings=[]
    for qp in quality_phrases:
        qp = qp.lower()
        if qp in embedding_model:
            embedding_idx_to_word[len(quality_embeddings)] = qp
            quality_embeddings.append(embedding_model[qp])
        else:
            print(qp)
            
    return quality_embeddings

In [45]:
# quality_phrases_train = get_quality_phrases(lines[:train_len])
# quality_phrases_test = get_quality_phrases(lines[train_len+1:])

In [56]:
quality_phrases_train_raw = [p.replace('_',' ') for p in quality_phrases_train]

In [57]:
len(quality_phrases_train_raw)

9950

In [58]:
# get the entire list
quality_phrases=set()
quality_phrases = quality_phrases.union(quality_phrases_train).union(quality_phrases_test)

In [59]:
len(quality_phrases)

8174

In [ ]:
quality_embeddings = get_quality_phrases_embeddings(quality_phrases)

In [61]:
len(quality_phrases)

8174

In [62]:
len(quality_embeddings)

3149

In [ ]:
### Make some predictions? -- using labeled data (directly test the matching phrase.. In pratice, should use n-gram instead)

In [ ]:
### convert to 2-gram, 3-gram and add to the Counter

In [63]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [50]:
## df_text has been preprocessed and ready to be used! -- oh no, it should be unsupervised, so do not need the phrase-to-token step
df_test = unprocessed_lines[20001:]
df_test = preprocess_df(df_test)

In [61]:
c = Counter(['black'])


In [67]:
c.update({"red",1})

In [64]:
from collections import defaultdict
from tqdm import tqdm

In [91]:
## Build n-gram vocabulary for df-test
## an inverted index might be needed? --- so if satisfy frequency, it will be encoded as a single token
## so in the loop, I need to create a n-gram dict(token, idx), a counter, check if it is in the high-quality corpus (quality_phrases_train_raw)
# counter_test = Counter()
# quality_phrase_val = set()
# inverted_index = defaultdict(lambda: list())
# for idx, article in tqdm(enumerate(df_test)):
#     n_grams = get_ngrams(article, 2)
#     temp = defaultdict(lambda:0)
#     for gram in n_grams:
#         if gram in quality_phrases_train_raw:
#             quality_phrase_val.add(gram)
#         temp[gram]+=1
#         inverted_index[gram].append(idx)

#     counter_test.update(temp)
    
        

970it [04:23,  3.68it/s]


In [65]:
### Senario 2: convert the whole article to bi-grams, save the candidates
counter_test = Counter()
quality_phrase_val = set()
inverted_index = defaultdict(lambda: list())
for idx, article in tqdm(enumerate(df_test)):
    n_grams = article.split(' ')
    temp = defaultdict(lambda:0)
    for gram in n_grams:
        if gram in quality_phrases_train:
            quality_phrase_val.add(gram)
        temp[gram]+=1
        inverted_index[gram].append(idx)

    counter_test.update(temp)

749it [00:13, 54.53it/s]


In [ ]:
# Inspecting electricity power lines
# Inspecting_electricity electricity_power power_lines

In [66]:
len(quality_phrase_val), len(counter_test)

(1186, 47912)

In [67]:
## Now check the counter and prepare candidate set: candidate = satisfy frequency requirment - intersection with quality phrase
## data structure for candidates: list()
thres = 5
candidates = [x for x,count in counter_test.items() if count >= thres and x not in quality_phrase_val]


In [68]:
len(candidates)

4539

In [121]:
### Now go back to the test corpus and replace the occurences of candidates with a single phrase token. 
## Use the inverted index

## This is for senario1: replace part of n-gram in the article only
# doc_retriver=defaultdict(lambda: set())
# for x in candidates:
#     for idx in inverted_index[x]:
#         doc_retriver[idx].add(x)

In [115]:
test = "beidan is beidan is best"
test.replace("beidan","huangbeidan")

'huangbeidan is huangbeidan is best'

In [116]:
# replace in the corpus
## attention: can't simply replace! there might be overlaps lmao....

# for idx, article in enumerate(df_test):
#     if idx not in doc_retriver: continue
#     phrase_to_replace = doc_retriver[idx]
#     for p in phrase_to_replace:
#         df_test[idx] = df_test[idx].replace(p, "_".join(p.split(' ')))

In [ ]:
## Senario 2: compute similarity vectors for the candidates

### Evaluation

In [69]:
## update the training model first
tagged_train_test = [word_tokenize(_d) for i, _d in enumerate(df_test)]


word_counts_c, vocabulary_c, vocabulary_inv_c = build_vocab(tagged_train_test)
inp_data_c = [[vocabulary_c[word] for word in text] for text in tagged_train_test]
new_sentences = [[vocabulary_inv_c[w] for w in s] for s in inp_data_c]


In [70]:
excount=0
for line in new_sentences:
    excount += 1

In [71]:
embedding_model.build_vocab(new_sentences, update=True)
embedding_model.train(new_sentences, total_examples = excount, epochs=3)

(325303, 487695)

In [ ]:
### Evaluate candidates
## 1.top similar item should be quality phrase 2.similarity should above certain threshold

In [72]:
candidates[:5]

['', '_the', 'the_utility', 'utility_model', 'model_proposes']

In [73]:
quality_temp=[]
for test_cand in tqdm(candidates):
    if test_cand not in embedding_model: continue
    top_matches = embedding_model.most_similar(positive=[test_cand])[:2]
    for match in top_matches:
        if match[0] not in quality_phrases_train or match[1] < 0.85: continue
        quality_temp.append(match)

  0%|          | 0/4539 [00:00<?, ?it/s]/home/beidan/miniconda3/envs/cse291a/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/beidan/miniconda3/envs/cse291a/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.
100%|██████████| 4539/4539 [00:11<00:00, 385.63it/s]


In [74]:
len(quality_temp), len(quality_phrase_val)

(657, 1186)

In [75]:
quality_temp_words = [p[0] for p in quality_temp]

In [76]:
surplus = [p for p in quality_temp_words if p not in quality_phrases_test]

In [77]:
len(set(surplus)), set(surplus)

(120,
 {'2nd_embodiment',
  '3gpp_ts',
  'access_nodes',
  'active_nn',
  'aerial_vehicles',
  'air_traffic',
  'almanac_data',
  'artificial_intelligence',
  'autonomous_operation',
  'blade_assemblies',
  'blanking_mouth',
  'block_chain',
  'block_diagram',
  'boom_spreader',
  'bull_stick',
  'cage_structure',
  'changing_mechanism',
  'charging_connector',
  'communication_interface',
  'compressed_air',
  'connecting_pin',
  'control_mainboard',
  'convex_stupefied',
  'coordinate_information',
  'cross_correlation',
  'depth_information',
  'detailed_description',
  'discharge_gate',
  'docking_mechanism',
  'driving_gear',
  'driving_unit',
  'driving_wheel',
  'edge_computing',
  'elastic_component',
  'electric_supply',
  'electricity_generation',
  'electrified_rail',
  'environmental_monitoring',
  'exemplary_embodiments',
  'exhaust_fans',
  'fixed_link',
  'focus_adjusting',
  'forced_convection',
  'gear_box',
  'guide_post',
  'handover_destination',
  'hijacking_messag

In [79]:
surplus_write = [p.replace('_',' ') for p in surplus]

In [80]:
## Active Learning Injection Part - add the surplus phrases to Active Learning expert label file
with open('input/patentdb-medium-expert.txt', 'w') as f:
    f.writelines("%s\n" % p for p in surplus_write)

In [351]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1,-1), feature_vec_2.reshape(1,-1))[0][0]

In [353]:
get_cosine_similarity(quality_embeddings[0],quality_embeddings[3])

0.3524833

In [24]:
embedding_model.most_similar(positive=['agitating_shaft'])

/home/beidan/miniconda3/envs/cse291a/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('gear_shaft', 0.9258540868759155),
 ('driven_shaft', 0.9127465486526489),
 ('with_rotary', 0.9122529029846191),
 ('circular_shaft', 0.9077884554862976),
 ('leading_screw', 0.9030115604400635),
 ('on_agitating', 0.9028359651565552),
 ('articulated_shaft', 0.9022598266601562),
 ('hollow_shaft', 0.8986348509788513),
 ('shaft_side', 0.8972604870796204),
 ('roll_shaft', 0.8969671130180359)]

In [455]:
test_article = tagged_test_data[3]
for token in test_article:
    if token in embedding_model:
        print(token)

embodiments
present_invention
disclose
kind
power_supply
solar_energy
unmanned
plane
power_supply
includes
power_supply
module
current_voltage
monitoring_modular
first
voltage_conversion_module
second
voltage_conversion_module
tertiary
voltage_conversion_module
voltage
power_supply
module
subjected
dc
dc
first
voltage_conversion_module
second
voltage_conversion_module
tertiary
voltage_conversion_module
converted
three
kinds
electric
signals
meets
requirement
unmanned_electromechanical_source
multi
output
hand
electric_current
exported
power_supply
module
current_voltage
monitoring_modular
one
hand
electric_current
input
first
voltage_conversion_module
second
voltage_conversion_module
carries
conversion
voltage
hand
electric_current
power_supply
module
output
monitored
ensure
circuit
safety


/home/beidan/miniconda3/envs/cse291a/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [383]:
##online word2vec???

In [390]:
# with open('test_c.txt', 'w') as f:
#     f.write(articles[20001])

In [430]:
# test_article.append('information_security')

In [449]:
word_counts_c, vocabulary_c, vocabulary_inv_c = build_vocab([test_article])
inp_data_c = [[vocabulary_c[word] for word in text] for text in [test_article]]
new_sentences = [[vocabulary_inv_c[w] for w in s] for s in inp_data_c]


In [450]:
excount=0
for line in new_sentences:
    excount += 1

In [451]:
embedding_model.build_vocab(new_sentences, update=True)

In [452]:
embedding_model.train(new_sentences, total_examples = excount, epochs=3)

(148, 213)

In [454]:
embedding_model['voltage_conversion_module']

/home/beidan/miniconda3/envs/cse291a/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.01287237,  0.0090122 , -0.00884458,  0.00848831,  0.00032224,
       -0.00666879, -0.0013857 , -0.00014764, -0.00687364,  0.00806865,
       -0.00601127, -0.00032863, -0.00272678, -0.01601487, -0.00389749,
        0.00042824,  0.00086173,  0.00985634,  0.00572525, -0.00297683,
       -0.01500027, -0.00444778, -0.00254812, -0.01314077,  0.01753565,
       -0.00735723,  0.00334592,  0.01917579,  0.00578594, -0.00970757,
        0.00142931, -0.01196378,  0.00858092, -0.00496101,  0.00096193,
       -0.00423299,  0.0003428 ,  0.00543864,  0.0143293 , -0.00043601,
        0.00637902,  0.00132579, -0.00044674, -0.0150325 ,  0.00677556,
        0.00469601, -0.00556763,  0.00105794,  0.00155691,  0.0041729 ,
       -0.00970957, -0.00418425,  0.00195676, -0.01023265, -0.00713241,
        0.0017502 ,  0.0062706 , -0.00058419,  0.00763606,  0.00431534,
       -0.00032861,  0.01195908, -0.00846418,  0.00086708, -0.00189516,
        0.00062416, -0.00613898, -0.01201752,  0.00125749,  0.00